Даны интегральый уравнения Фредгольма и Вольтеррра 2-го рода при следующих значениях их параметров.  


| $ K(x,s) $ | $ f(x) $ | $ \lambda $ | $ [a,b] $ |
|------------|----------|-------------|-----------|
| $ e^{x-s} $ | $ e^{x-2}(e-1) $ | 0.3 | $ [0,\frac{\pi}{2}] $ |

Необходимо:  
1. Методом механических квадратур найти приближенные решения при N=10, используя составную формулу Симпсона для ИУФ-2 и формулу правых прямоугольников для ИУВ-2.  
2. Методом последовательных приближений при n=5, найти приближенные решения ИУФ-2 и ИУВ-2. Для вычисления интегралов применить квадратурные формулы, указаные в п.1. Оценить погрешность.  
3. Провести сравнительный анализ решений, полученных в п.п. 1, 2 результатов. Вычислить по каждому методу решение в точке $ x^*=\frac{a+b}{2.2} $ и объяснить разницу.  

In [126]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
def get_quad_coefs(a, b, N, method):
    h = (b-a)/N
    coefs = []
    
    if method == 'r_rectangles':
        coefs.append(0)
        for i in range(N):
            coefs.append(h)
            
    elif method == 'simpson':
        for i in range(N+1):
            if(i == 0) or i == N:
                coefs.append(h/3)
            elif i%2 == 0:
                coefs.append(2*h/3)
            elif i%2 == 1:
                coefs.append(4*h/3)
    
    return np.array(coefs)

In [150]:
# параметры из условия
def K(x,s):
    return np.exp(x-s)

def K_volt(x, s):
    return np.exp(x-s) * (s <= x)
    
def f(x):
    return np.exp(x-2)*(np.exp(1) - 1)

lmb = 0.1
a = 0
b = np.pi / 2

In [151]:
# Метод механических квадратур

def mech_quad(K, f, lmb, a, b, N, method):
    h = (b-a)/N
    x = np.array([a + h*i for i in range(N+1)])

    coefs = get_quad_coefs(a, b, N, method)
    
    A = np.zeros((N+1, N+1)) # матрица системы
    b = np.zeros((N+1, 1)) # неоднородность системы
    for i in range(N+1):
        b[i] = f(x[i])
        for j in range(N+1):
            A[i][j] = 1 - lmb* coefs[j]*K(x[i], x[i]) if i == j else -lmb*coefs[j]*K(x[i], x[j])
    u =np.linalg.solve(A,b).flatten()
    return x, u, coefs

def get_solution_at_point(x,u,quad_coefs, lmb,K, f, x0):
    u_0 = 0.0
    for x_i, y_i, A_i in zip(x, u, quad_coefs):
        u_0 += lmb * A_i * K(x0, x_i) * y_i
    
    u_0 += f(x0)
    return u_0

In [153]:
print('ИУф-2')
mq_solution_iuf = mech_quad(K, f, lmb, a, b, 10, 'simpson')
for i in range(mq_solution_iuf[0].shape[0]):
    print(f'u({mq_solution_iuf[0][i]:.2f}) = {mq_solution_iuf[1][i]:.6f}')

print(f'u(a+b/2.2) = u({(a+b)/2:.2f})={get_solution_at_point(mq_solution_iuf[0], mq_solution_iuf[1], mq_solution_iuf[2], lmb, K, f, (a+b)/2):.6f}')
print()
print('ИУВ-2')
mq_solution_iuv = mech_quad(K_volt, f, lmb, a, b, 10, 'r_rectangles')
for i in range(mq_solution_iuv[0].shape[0]):
    print(f'u({mq_solution_iuv[0][i]:.2f}) = {mq_solution_iuv[1][i]:.6f}')
print(f'u(a+b/2.2) = u({(a+b)/2:.2f})={get_solution_at_point(mq_solution_iuv[0], mq_solution_iuv[1], mq_solution_iuv[2], lmb, K_volt, f, (a+b)/2):.6f}')
print()

ИУф-2
u(0.00) = 0.275879
u(0.16) = 0.322803
u(0.31) = 0.377708
u(0.47) = 0.441952
u(0.63) = 0.517123
u(0.79) = 0.605080
u(0.94) = 0.707998
u(1.10) = 0.828420
u(1.26) = 0.969325
u(1.41) = 1.134196
u(1.57) = 1.327110
u(a+b/2.2) = u(0.79)=0.605080

ИУВ-2
u(0.00) = 0.232544
u(0.16) = 0.276440
u(0.31) = 0.328621
u(0.47) = 0.390652
u(0.63) = 0.464392
u(0.79) = 0.552052
u(0.94) = 0.656258
u(1.10) = 0.780134
u(1.26) = 0.927394
u(1.41) = 1.102450
u(1.57) = 1.310551
u(a+b/2.2) = u(0.79)=0.552052



In [122]:
def iterations_method(K, f, lmb, a, b, N_splits, N_it, method):
    h = (b-a)/N_splits
    x = np.array([a + h*i for i in range(N_splits+1)])
    coefs = get_quad_coefs(a, b, N_splits, method)
    phis = np.zeros((N_it+1, N_splits+1)) # (i, j) -- элемент y_i(x_j)
    
    for i in range(N_it+1):
        for j in range(N_splits+1):
            if i == 0:
                phis[i][j] = f(x[j])
            else:
                phis[i][j] = np.sum([ coefs[k]*K(x[j], x[k])*phis[i-1][k] for k in range(N_splits + 1)])
    
    u = np.zeros((N_splits + 1, ))
    for i in range(N_splits + 1):
        for j in range(N_it + 1):
            u[i] += lmb**j * phis[j][i]
    return x, u, coefs

In [147]:
print('ИУф-2')
mpp_solution_iuf = iterations_method(K, f, lmb, a, b, 10,5 ,'simpson')
for i in range(solution[0].shape[0]):
    print(f'u({solution[0][i]:.2f}) = {solution[1][i]:.6f}')

print(f'u(a+b/2.2) = u({(a+b)/2:.2f})={get_solution_at_point(mpp_solution_iuf[0], mpp_solution_iuf[1], mpp_solution_iuf[2], lmb, K, f, (a+b)/2):.6f}')
print()


ИУф-2
u(0.00) = 0.232544
u(0.16) = 0.276440
u(0.31) = 0.328621
u(0.47) = 0.390652
u(0.63) = 0.464392
u(0.79) = 0.552052
u(0.94) = 0.656258
u(1.10) = 0.780134
u(1.26) = 0.927394
u(1.41) = 1.102450
u(1.57) = 1.310551
u(a+b/2.2) = u(0.79)=0.605079



In [148]:
print('ИУВ-2')
mpp_solution_iuv = iterations_method(K_volt, f, lmb, a, b, 10,5 ,'r_rectangles')
for i in range(solution[0].shape[0]):
    print(f'u({solution[0][i]:.2f}) = {solution[1][i]:.6f}')

print(f'u(a+b/2.2) = u({(a+b)/2:.2f})={get_solution_at_point(mpp_solution_iuv[0], mpp_solution_iuv[1], mpp_solution_iuv[2], lmb, K, f, (a+b)/2):.6f}')
print()

ИУВ-2
u(0.00) = 0.232544
u(0.16) = 0.276440
u(0.31) = 0.328621
u(0.47) = 0.390652
u(0.63) = 0.464392
u(0.79) = 0.552052
u(0.94) = 0.656258
u(1.10) = 0.780134
u(1.26) = 0.927394
u(1.41) = 1.102450
u(1.57) = 1.310551
u(a+b/2.2) = u(0.79)=0.597530



Построим графики полученных функций